# 用嵌入和对话模型制作一个简单的内部文档助手

In [ ]:
import openai
import pandas as pd
import numpy as np
import json

with open("../config.json") as f:
    config = json.load(f)


openai.api_key = config["api_key"]
openai.api_base = config["api_base"]
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
chat_model = "gpt-35-turbo"
embed_model = "text-embedding-ada-002"

创建 prompt 模板

In [ ]:
prompt_prefix = """
你是一个客服，回答用户关于文档的问题。
仅使用以下资料提供的事实进行回答。 如果下面没有足够的信息，就说你不知道。不要生成不使用以下资料的答案。

资料：
{sources}
"""
summary_prompt_template = """
以上是到目前为止的对话记录，下面我将提出一个新问题，需要通过在知识库中搜索相关的条目来回答问题。根据以上的对话记录和下面的新问题，生成一个英文的查询语句，用于在知识库中搜索相关的条目。
注意，你只需要回复查询语句，不要加其他任何前缀或后缀。

新问题：
{question}
"""

定义辅助函数

In [ ]:
def get_chat_answer(messages, max_token=1024, stream=False):
    if stream:
        return openai.ChatCompletion.create(
            engine=chat_model,
            messages=messages,
            temperature=0.7,
            max_tokens=max_token,
            stream=True,
        )
    return openai.ChatCompletion.create(
        engine=chat_model,
        messages=messages,
        temperature=0.7,
        max_tokens=max_token,
        stream=False,
    )["choices"][0]["text"]


def get_embedding(text):
    return openai.Embedding.create(
        engine=embed_model,
        input=text,
    )["data"][0]["embedding"]


def cos_sim(a, b):
    return np.dot(a, b)


docs = pd.read_csv("docs.csv", converters={"embedding": eval})
pd.set_option("display.max_colwidth", None) # 取消列宽限制
history = []


In [ ]:
user_input = ""

if len(history) == 0:
    query = user_input
else:
    query = get_chat_answer(
        history
        + [
            {
                "role": "user",
                "content": summary_prompt_template.format(question=user_input),
            }
        ],
        max_token=32,
    )["content"]

print(f"[Searching: {query}]")
query_embedding = get_embedding(query)
dot_products = np.dot(np.stack(docs["embedding"].values), query_embedding)
top_index = np.argsort(dot_products)[-1:]
top_content = (
    docs.iloc[top_index]["content"].to_string(index=False)
    if dot_products[top_index] > 0.8
    else "no information"
)
history.append({"role": "user", "content": user_input})

massage = [
    {"role": "user", "content": prompt_prefix.format(sources=top_content)},
    {
        "role": "assistant",
        "content": "好的，我只会根据以上提供的资料提供的内容回答问题，我不会回答不使用资源的内容。",
    },
] + history

res = get_chat_answer(massage, stream=True)
next(res)
content = ""
while txt := next(res).choices[0].delta:
    text = txt["content"]
    print(text, end="")
    content += text

history.append({"role": "assistant", "content": content})
print("\n" + "-" * 50, end="\n\n")
